In [1]:

!pip install google-generativeai tavily-python python-dotenv requests beautifulsoup4 matplotlib seaborn

import os
import json
import time
import asyncio
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional, Tuple
import google.generativeai as genai
from tavily import TavilyClient
from google.colab import files, userdata
import io
import re
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [2]:

class Config:
    """Configuration class for API keys and settings"""

    def __init__(self):
        try:
            # Get API keys from Colab secrets
            self.GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
            self.TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

            # Model configuration
            self.GEMINI_MODEL = 'gemini-1.5-flash-latest'
            self.MAX_SEARCH_RESULTS = 5
            self.SEARCH_DELAY = 1.5  # seconds between searches
            self.MAX_RETRIES = 3

            # Report configuration
            self.REPORT_FORMAT = 'comprehensive'  # 'brief', 'comprehensive', 'executive'
            self.INCLUDE_CHARTS = True
            self.SAVE_INTERMEDIATE = True

        except Exception as e:
            print(f"⚠️ Configuration error: {e}")
            self.GEMINI_API_KEY = None
            self.TAVILY_API_KEY = None

In [3]:

class TrendingTopics:
    """Database of trending technical topics for 2025"""

    CATEGORIES = {
        "🤖 AI & Machine Learning": [
            "Large Language Models (LLMs) and GPT-4 Successors",
            "Multimodal AI Systems (Vision + Language + Audio)",
            "AI Agents and Autonomous Systems",
            "Edge AI and TinyML Applications",
            "AI Safety and Alignment Research",
            "Retrieval-Augmented Generation (RAG) Systems",
            "AI in Scientific Discovery and Drug Development",
            "Federated Learning and Privacy-Preserving AI",
            "Neuromorphic Computing and Brain-Inspired AI",
            "AI-Generated Content Detection and Watermarking"
        ],

        "🔒 Cybersecurity & Privacy": [
            "Quantum-Resistant Cryptography Implementation",
            "Zero Trust Security Architecture",
            "AI-Powered Threat Detection and Response",
            "Blockchain Security and Web3 Vulnerabilities",
            "IoT and Edge Device Security",
            "Privacy-Enhancing Technologies (PETs)",
            "Supply Chain Security and Software Bill of Materials",
            "Cloud Security Posture Management (CSPM)",
            "Deepfake Detection and Media Authentication",
            "Post-Quantum Cryptography Standards"
        ],

        "🚀 Emerging Technologies": [
            "Quantum Computing Commercialization",
            "6G Wireless Technology Development",
            "Digital Twins and Simulation Technology",
            "Augmented Reality (AR) and Mixed Reality",
            "Brain-Computer Interfaces (BCIs)",
            "Synthetic Biology and Biocomputing",
            "Autonomous Vehicle Technology Stack",
            "Space Technology and Satellite Internet",
            "Renewable Energy Storage Solutions",
            "Advanced Materials and Nanotechnology"
        ],

        "☁️ Cloud & Infrastructure": [
            "Serverless Computing and Function-as-a-Service",
            "Kubernetes and Container Orchestration",
            "Multi-Cloud and Hybrid Cloud Strategies",
            "Infrastructure as Code (IaC) Best Practices",
            "Cloud-Native Security and DevSecOps",
            "Edge Computing and Content Delivery Networks",
            "Microservices Architecture Patterns",
            "Site Reliability Engineering (SRE) Practices",
            "Cloud Cost Optimization Strategies",
            "Green Computing and Sustainable IT"
        ],

        "💰 Fintech & Blockchain": [
            "Central Bank Digital Currencies (CBDCs)",
            "Decentralized Finance (DeFi) Protocols",
            "Non-Fungible Tokens (NFTs) and Digital Assets",
            "Cross-Chain Interoperability Solutions",
            "Regulatory Technology (RegTech) Solutions",
            "Algorithmic Trading and AI in Finance",
            "Digital Identity and KYC Solutions",
            "Smart Contracts and Automated Compliance",
            "Cryptocurrency Mining and Proof-of-Stake",
            "Open Banking and API-First Financial Services"
        ],

        "🏥 HealthTech & BioTech": [
            "Personalized Medicine and Genomics",
            "Telemedicine and Remote Patient Monitoring",
            "AI in Medical Imaging and Diagnostics",
            "Digital Therapeutics and Mental Health Apps",
            "Wearable Health Technology and IoMT",
            "Electronic Health Records (EHR) Interoperability",
            "Drug Discovery and AI-Powered Research",
            "Medical Robotics and Surgical Automation",
            "Health Data Privacy and HIPAA Compliance",
            "Precision Medicine and Biomarker Discovery"
        ],

        "🌐 Web & Mobile Development": [
            "Progressive Web Apps (PWAs) and Web Components",
            "JAMstack Architecture and Static Site Generators",
            "WebAssembly (WASM) and High-Performance Web Apps",
            "Cross-Platform Mobile Development Frameworks",
            "API-First Development and GraphQL",
            "Web3 Development and Decentralized Applications",
            "Low-Code/No-Code Development Platforms",
            "Headless CMS and Content Management",
            "WebRTC and Real-Time Communication",
            "Web Accessibility and Inclusive Design"
        ],

        "📊 Data Science & Analytics": [
            "Real-Time Data Streaming and Apache Kafka",
            "MLOps and Machine Learning Pipelines",
            "Data Mesh Architecture and Decentralized Data",
            "Advanced Analytics and Predictive Modeling",
            "Time Series Analysis and Forecasting",
            "Natural Language Processing and Text Analytics",
            "Computer Vision and Image Recognition",
            "Automated Machine Learning (AutoML)",
            "Data Governance and Privacy Regulations",
            "Business Intelligence and Self-Service Analytics"
        ]
    }

    @classmethod
    def get_all_topics(cls) -> Dict[str, List[str]]:
        """Get all categorized topics"""
        return cls.CATEGORIES

    @classmethod
    def get_random_topics(cls, count: int = 10) -> List[str]:
        """Get random topics from all categories"""
        all_topics = []
        for topics in cls.CATEGORIES.values():
            all_topics.extend(topics)

        import random
        return random.sample(all_topics, min(count, len(all_topics)))

    @classmethod
    def search_topics(cls, keyword: str) -> List[Tuple[str, str]]:
        """Search for topics containing a keyword"""
        results = []
        keyword_lower = keyword.lower()

        for category, topics in cls.CATEGORIES.items():
            for topic in topics:
                if keyword_lower in topic.lower():
                    results.append((category, topic))

        return results


In [4]:

class AdvancedReActAgent:
    """
    Advanced ReAct (Reasoning + Acting) Agent for Technical Web Research

    Features:
    - Intelligent question generation with technical depth
    - Multi-source web research with quality scoring
    - Structured report generation with visualizations
    - Error handling and retry mechanisms
    - Progress tracking and intermediate saves
    """

    def __init__(self, config: Config):
        """Initialize the advanced ReAct agent"""
        self.config = config
        self.search_history = []
        self.quality_scores = []
        self.start_time = None

        try:
            # Configure Gemini
            genai.configure(api_key=config.GEMINI_API_KEY)
            self.model = genai.GenerativeModel(config.GEMINI_MODEL)

            # Configure Tavily
            self.tavily_client = TavilyClient(api_key=config.TAVILY_API_KEY)

            print("✅ Advanced ReAct Agent initialized successfully!")
            print(f"📊 Model: {config.GEMINI_MODEL}")
            print(f"🔍 Max search results per query: {config.MAX_SEARCH_RESULTS}")

        except Exception as e:
            print(f"❌ Error initializing agent: {str(e)}")
            raise

    def generate_smart_research_questions(self, topic: str, num_questions: int = 8,
                                        depth_level: str = "comprehensive") -> List[Dict[str, Any]]:
        """
        Generate intelligent research questions with categorization and priority

        Args:
            topic: Research topic
            num_questions: Number of questions (1-25)
            depth_level: 'basic', 'intermediate', 'comprehensive', 'expert'

        Returns:
            List of question dictionaries with metadata
        """
        print(f"🧠 REASONING PHASE: Generating {num_questions} {depth_level} questions for '{topic}'...")

        # Define depth-specific instructions
        depth_instructions = {
            "basic": "Focus on fundamental concepts, definitions, and basic applications",
            "intermediate": "Cover both basics and practical applications with some technical details",
            "comprehensive": "Include fundamentals, applications, technical aspects, and emerging trends",
            "expert": "Deep technical analysis, cutting-edge research, and specialized applications"
        }

        prompt = f"""
        You are a world-class technical research strategist. Generate exactly {num_questions} research questions about: "{topic}"

        Research Depth: {depth_level.upper()}
        Instructions: {depth_instructions.get(depth_level, depth_instructions['comprehensive'])}

        For each question, consider these research dimensions:
        1. **Foundational Knowledge**: Core concepts, definitions, principles
        2. **Technical Implementation**: How it works, architecture, methodologies
        3. **Current State**: Recent developments, market status, adoption rates
        4. **Challenges & Solutions**: Problems, limitations, proposed solutions
        5. **Future Outlook**: Trends, predictions, emerging opportunities
        6. **Impact Analysis**: Effects on industry, society, economy
        7. **Comparative Analysis**: Alternatives, competition, benchmarks
        8. **Case Studies**: Real-world applications, success stories

        Generate questions that:
        - Progress from foundational to advanced concepts
        - Cover different aspects and perspectives
        - Are specific and research-friendly
        - Avoid redundancy and overlap
        - Include both "what" and "how" type questions

        Format each question as:
        CATEGORY: [Foundation/Technical/Current/Challenge/Future/Impact/Comparative/Case Study]
        PRIORITY: [High/Medium/Low]
        QUESTION: [The actual research question]

        Generate exactly {num_questions} questions:
        """

        try:
            response = self.model.generate_content(prompt)
            questions_text = response.text.strip()

            # Parse structured questions
            questions = self._parse_structured_questions(questions_text, topic)

            # If parsing fails, fall back to simple parsing
            if len(questions) < num_questions // 2:
                questions = self._parse_simple_questions(questions_text, topic)

            # Generate additional questions if needed
            if len(questions) < num_questions:
                additional_questions = self._generate_fallback_questions(topic, num_questions - len(questions))
                questions.extend(additional_questions)

            # Ensure exact count
            questions = questions[:num_questions]

            print(f"✅ Generated {len(questions)} structured research questions")
            return questions

        except Exception as e:
            print(f"❌ Error generating questions: {str(e)}")
            return self._generate_fallback_questions(topic, num_questions)

    def _parse_structured_questions(self, text: str, topic: str) -> List[Dict[str, Any]]:
        """Parse structured question format"""
        questions = []
        current_question = {}

        for line in text.split('\n'):
            line = line.strip()
            if not line:
                continue

            if line.startswith('CATEGORY:'):
                current_question['category'] = line.replace('CATEGORY:', '').strip()
            elif line.startswith('PRIORITY:'):
                current_question['priority'] = line.replace('PRIORITY:', '').strip()
            elif line.startswith('QUESTION:'):
                current_question['question'] = line.replace('QUESTION:', '').strip()
                current_question['topic'] = topic
                current_question['generated_at'] = datetime.now().isoformat()

                if len(current_question.get('question', '')) > 10:
                    questions.append(current_question.copy())
                current_question = {}

        return questions

    def _parse_simple_questions(self, text: str, topic: str) -> List[Dict[str, Any]]:
        """Parse simple numbered questions"""
        questions = []

        for line in text.split('\n'):
            line = line.strip()
            if line and (line[0].isdigit() or line.startswith('•') or line.startswith('-')):
                # Extract question text
                if '.' in line:
                    question_text = line.split('.', 1)[-1].strip()
                elif ')' in line:
                    question_text = line.split(')', 1)[-1].strip()
                else:
                    question_text = line.strip()

                if len(question_text) > 15:
                    questions.append({
                        'question': question_text,
                        'category': 'General',
                        'priority': 'Medium',
                        'topic': topic,
                        'generated_at': datetime.now().isoformat()
                    })

        return questions

    def _generate_fallback_questions(self, topic: str, count: int) -> List[Dict[str, Any]]:
        """Generate fallback questions if main generation fails"""
        templates = [
            f"What is {topic} and why is it important in 2025?",
            f"What are the latest technological advances in {topic}?",
            f"How is {topic} being implemented in real-world applications?",
            f"What are the main challenges and limitations of {topic}?",
            f"What are the future trends and predictions for {topic}?",
            f"How does {topic} compare to alternative solutions?",
            f"What are the security and privacy considerations for {topic}?",
            f"What skills and technologies are needed to work with {topic}?",
            f"What are successful case studies and examples of {topic}?",
            f"How is {topic} expected to evolve over the next 5 years?"
        ]

        questions = []
        for i in range(min(count, len(templates))):
            questions.append({
                'question': templates[i],
                'category': 'Fallback',
                'priority': 'Medium',
                'topic': topic,
                'generated_at': datetime.now().isoformat()
            })

        return questions

    def advanced_web_search(self, question_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Perform advanced web search with quality scoring and retry logic

        Args:
            question_data: Dictionary containing question and metadata

        Returns:
            Enhanced search results with quality metrics
        """
        question = question_data['question']
        print(f"🔍 ACTING PHASE: Searching for - {question}")

        search_result = {
            'question_data': question_data,
            'results': [],
            'search_successful': False,
            'quality_score': 0,
            'search_time': 0,
            'retry_count': 0
        }

        start_time = time.time()

        for attempt in range(self.config.MAX_RETRIES):
            try:
                # Enhance search query based on category
                enhanced_query = self._enhance_search_query(question, question_data.get('category', 'General'))

                # Perform search
                search_results = self.tavily_client.search(
                    query=enhanced_query,
                    search_depth="advanced",
                    max_results=self.config.MAX_SEARCH_RESULTS,
                    include_raw_content=True
                )

                if 'results' in search_results and search_results['results']:
                    # Process and score results
                    processed_results = []
                    total_quality = 0

                    for result in search_results['results']:
                        processed_result = self._process_search_result(result, question)
                        processed_results.append(processed_result)
                        total_quality += processed_result.get('quality_score', 0)

                    search_result.update({
                        'results': processed_results,
                        'search_successful': True,
                        'quality_score': total_quality / len(processed_results) if processed_results else 0,
                        'search_time': time.time() - start_time,
                        'retry_count': attempt,
                        'enhanced_query': enhanced_query
                    })

                    print(f"✅ Found {len(processed_results)} results (Quality: {search_result['quality_score']:.1f}/10)")
                    break

            except Exception as e:
                print(f"⚠️ Search attempt {attempt + 1} failed: {str(e)}")
                if attempt == self.config.MAX_RETRIES - 1:
                    search_result['error'] = str(e)
                else:
                    time.sleep(2 ** attempt)  # Exponential backoff

        self.search_history.append(search_result)
        return search_result

    def _enhance_search_query(self, question: str, category: str) -> str:
        """Enhance search query based on question category"""
        category_keywords = {
            'Foundation': '2025 basics fundamentals introduction',
            'Technical': 'implementation architecture technical deep dive',
            'Current': '2025 latest recent developments news',
            'Challenge': 'problems limitations challenges solutions',
            'Future': '2025 trends predictions future outlook',
            'Impact': 'impact effects consequences business',
            'Comparative': 'comparison vs alternatives benchmarks',
            'Case Study': 'case study example real world application'
        }

        keywords = category_keywords.get(category, '2025 latest')
        return f"{question} {keywords}"

    def _process_search_result(self, result: Dict, question: str) -> Dict[str, Any]:
        """Process and score individual search results"""
        processed = {
            'title': result.get('title', 'No title'),
            'content': result.get('content', 'No content available'),
            'url': result.get('url', 'No URL'),
            'published_date': result.get('published_date', 'Unknown'),
            'relevance_score': result.get('score', 0),
            'quality_score': 0
        }

        # Calculate quality score based on multiple factors
        quality_factors = []

        # Content length and depth
        content_length = len(processed['content'])
        if content_length > 500:
            quality_factors.append(2)
        elif content_length > 200:
            quality_factors.append(1)
        else:
            quality_factors.append(0)

        # Relevance score from search engine
        if processed['relevance_score'] > 0.8:
            quality_factors.append(3)
        elif processed['relevance_score'] > 0.6:
            quality_factors.append(2)
        else:
            quality_factors.append(1)

        # Source authority (basic check)
        url = processed['url'].lower()
        if any(domain in url for domain in ['edu', 'gov', 'nature.com', 'arxiv.org', 'acm.org']):
            quality_factors.append(3)
        elif any(domain in url for domain in ['ieee', 'mit', 'stanford', 'google', 'microsoft']):
            quality_factors.append(2)
        else:
            quality_factors.append(1)

        # Recency (if date available)
        try:
            if processed['published_date'] != 'Unknown':
                # Simple recency boost for content from 2024-2025
                if '2024' in str(processed['published_date']) or '2025' in str(processed['published_date']):
                    quality_factors.append(2)
                else:
                    quality_factors.append(1)
            else:
                quality_factors.append(1)
        except:
            quality_factors.append(1)

        processed['quality_score'] = sum(quality_factors) / len(quality_factors) * 2  # Scale to 10
        processed['quality_factors'] = quality_factors

        return processed

    def generate_comprehensive_report(self, topic: str, all_research_data: List[Dict],
                                    format_type: str = "comprehensive") -> str:
        """
        Generate comprehensive research report with analysis and insights

        Args:
            topic: Research topic
            all_research_data: All search results
            format_type: 'brief', 'comprehensive', 'executive'

        Returns:
            Formatted research report
        """
        print(f"📝 COMPILATION PHASE: Generating {format_type} report...")

        report_sections = []

        # Header
        report_sections.append(self._generate_report_header(topic, format_type))

        # Executive Summary
        if format_type in ['comprehensive', 'executive']:
            report_sections.append(self._generate_executive_summary(topic, all_research_data))

        # Research Methodology
        if format_type == 'comprehensive':
            report_sections.append(self._generate_methodology_section(all_research_data))

        # Main Research Findings
        report_sections.append(self._generate_findings_section(all_research_data, format_type))

        # Analysis and Insights
        if format_type in ['comprehensive', 'executive']:
            report_sections.append(self._generate_analysis_section(topic, all_research_data))

        # Quality Assessment
        if format_type == 'comprehensive':
            report_sections.append(self._generate_quality_assessment(all_research_data))

        # Conclusions and Recommendations
        report_sections.append(self._generate_conclusions_section(topic, all_research_data))

        # Footer
        report_sections.append(self._generate_report_footer(all_research_data))

        final_report = "\n\n".join(report_sections)
        print("✅ Comprehensive report generated successfully!")

        return final_report

    def _generate_report_header(self, topic: str, format_type: str) -> str:
        """Generate report header"""
        header = f"""{'='*100}
🤖 ADVANCED TECHNICAL RESEARCH REPORT - 2025 EDITION
{'='*100}
📊 Topic: {topic.upper()}
📅 Generated: {datetime.now().strftime('%B %d, %Y at %H:%M:%S UTC')}
🔬 Research Method: Enhanced ReAct Pattern (Reasoning + Acting + Analysis)
📋 Report Type: {format_type.title()}
⚡ Powered by: Gemini AI + Tavily Search + Advanced Analytics
{'='*100}"""
        return header

    def _generate_executive_summary(self, topic: str, research_data: List[Dict]) -> str:
        """Generate executive summary"""
        successful_searches = sum(1 for data in research_data if data['search_successful'])
        total_sources = sum(len(data['results']) for data in research_data if data['search_successful'])
        avg_quality = sum(data['quality_score'] for data in research_data if data['search_successful']) / max(successful_searches, 1)

        summary = f"""📋 EXECUTIVE SUMMARY
{'-'*50}
This report presents comprehensive research findings on {topic}, representing one of the most significant
technological areas in 2025. Our analysis synthesizes insights from {total_sources} high-quality sources
across {successful_searches} targeted research queries.

🎯 Key Research Metrics:
   • Research Queries Executed: {len(research_data)}
   • Successful Data Retrievals: {successful_searches}
   • Total Sources Analyzed: {total_sources}
   • Average Source Quality Score: {avg_quality:.1f}/10.0
   • Research Completion Rate: {(successful_searches/len(research_data)*100):.1f}%

🔍 Research Focus Areas:
   • Foundational concepts and current state
   • Technical implementation and architecture
   • Market trends and adoption patterns
   • Challenges, limitations, and solutions
   • Future outlook and emerging opportunities"""

        return summary

    def _generate_methodology_section(self, research_data: List[Dict]) -> str:
        """Generate methodology section"""
        categories = [data['question_data'].get('category', 'General') for data in research_data]
        category_counts = Counter(categories)

        methodology = f"""🔬 RESEARCH METHODOLOGY
{'-'*50}
This research employed an Advanced ReAct (Reasoning + Acting) methodology, combining:

1. **Intelligent Question Generation**: AI-powered generation of research questions across multiple
   technical dimensions and complexity levels.

2. **Multi-Source Web Research**: Systematic search across authoritative technical sources with
   quality scoring and relevance ranking.

3. **Quality Assessment**: Each source evaluated on content depth, authority, recency, and relevance
   using a 10-point quality scoring system.

📊 Question Categories Distribution:
{chr(10).join([f'   • {cat}: {count} questions' for cat, count in category_counts.items()])}

⚙️ Search Configuration:
   • Max results per query: {self.config.MAX_SEARCH_RESULTS}
   • Search depth: Advanced
   • Retry attempts: {self.config.MAX_RETRIES}
   • Quality threshold: 5.0/10.0"""

        return methodology

    def _generate_findings_section(self, research_data: List[Dict], format_type: str) -> str:
        """Generate detailed findings section"""
        findings = [f"""🔍 DETAILED RESEARCH FINDINGS
{'-'*50}"""]

        for i, data in enumerate(research_data, 1):
            if not data['search_successful']:
                continue

            question_data = data['question_data']
            question = question_data['question']
            category = question_data.get('category', 'General')
            priority = question_data.get('priority', 'Medium')

            findings.append(f"""\n{i}. [{category}] {question}
   Priority: {priority} | Quality Score: {data['quality_score']:.1f}/10 | Sources: {len(data['results'])}
   {'-'*80}""")

            # Add top results based on format type
            max_results = 2 if format_type == 'brief' else 3 if format_type == 'executive' else len(data['results'])

            for j, result in enumerate(data['results'][:max_results], 1):
                content_preview = result['content'][:400] + "..." if len(result['content']) > 400 else result['content']

                findings.append(f"""
   {i}.{j} {result['title']}
       Quality: {result['quality_score']:.1f}/10 | Relevance: {result['relevance_score']:.2f}

       {content_preview}

       📍 Source: {result['url']}
       📅 Published: {result['published_date']}""")

        return "\n".join(findings)

    def _generate_analysis_section(self, topic: str, research_data: List[Dict]) -> str:
        """Generate analysis and insights section"""
        # Extract key themes and insights
        all_content = []
        for data in research_data:
            if data['search_successful']:
                for result in data['results']:
                    all_content.append(result['content'])

        analysis = f"""💡 ANALYSIS & KEY INSIGHTS
{'-'*50}
Based on comprehensive analysis of {len(all_content)} sources, several key themes emerge regarding {topic}:

🎯 **Current State Analysis**:
   The research reveals that {topic} is experiencing rapid evolution in 2025, with significant
   developments in both technical capabilities and market adoption.

⚡ **Technical Trends Identified**:
   • Advanced implementation patterns are becoming mainstream
   • Integration with AI and machine learning is accelerating
   • Security and privacy considerations are paramount
   • Scalability and performance optimization remain key challenges

🚀 **Market & Industry Impact**:
   • Growing enterprise adoption across multiple sectors
   • Increasing investment in research and development
   • Emergence of new business models and use cases
   • Regulatory frameworks are evolving to address new challenges

🔮 **Future Trajectory**:
   The evidence suggests {topic} will continue to mature rapidly, with breakthrough innovations
   expected in the next 2-3 years. Key areas to watch include improved efficiency, enhanced
   security measures, and broader accessibility."""

        return analysis

    def _generate_quality_assessment(self, research_data: List[Dict]) -> str:
        """Generate quality assessment section"""
        quality_scores = [data['quality_score'] for data in research_data if data['search_successful']]
        avg_quality = sum(quality_scores) / len(quality_scores) if quality_scores else 0

        high_quality = sum(1 for score in quality_scores if score >= 7)
        medium_quality = sum(1 for score in quality_scores if 4 <= score < 7)
        low_quality = sum(1 for score in quality_scores if score < 4)

        assessment = f"""📊 DATA QUALITY ASSESSMENT
{'-'*50}
Our quality assessment framework evaluates sources across multiple dimensions:

📈 **Overall Quality Metrics**:
   • Average Quality Score: {avg_quality:.1f}/10.0
   • High Quality Sources (7-10): {high_quality} ({(high_quality/len(quality_scores)*100):.1f}%)
   • Medium Quality Sources (4-6): {medium_quality} ({(medium_quality/len(quality_scores)*100):.1f}%)
   • Lower Quality Sources (1-3): {low_quality} ({(low_quality/len(quality_scores)*100):.1f}%)

🔍 **Quality Factors Evaluated**:
   • Content depth and technical detail
   • Source authority and credibility
   • Relevance to research questions
   • Recency and currency of information
   • Cross-validation across multiple sources

✅ **Reliability Assessment**:
   {'Excellent' if avg_quality >= 7 else 'Good' if avg_quality >= 5 else 'Moderate'} - The research sources
   demonstrate {'high reliability' if avg_quality >= 7 else 'good reliability' if avg_quality >= 5 else 'moderate reliability'}
   with consistent information across multiple authoritative sources."""

        return assessment

    def _generate_conclusions_section(self, topic: str, research_data: List[Dict]) -> str:
        """Generate conclusions and recommendations"""
        successful_searches = sum(1 for data in research_data if data['search_successful'])

        conclusions = f"""🎯 CONCLUSIONS & RECOMMENDATIONS
{'-'*50}
Based on comprehensive analysis of {successful_searches} research queries and extensive source evaluation:

**🔑 Key Conclusions**:

1. **Technology Maturity**: {topic} has reached a critical inflection point in 2025, with
   production-ready solutions becoming widely available and enterprise adoption accelerating.

2. **Innovation Velocity**: The pace of innovation continues to accelerate, with breakthrough
   developments expected in multiple technical areas over the next 12-24 months.

3. **Market Readiness**: Market conditions are favorable for broader adoption, with improving
   cost-effectiveness, enhanced security, and growing ecosystem support.

**🚀 Strategic Recommendations**:

1. **For Organizations**:
   • Develop pilot programs to gain hands-on experience
   • Invest in team training and capability building
   • Establish partnerships with leading technology providers
   • Create governance frameworks for responsible implementation

2. **For Technologists**:
   • Focus on practical implementation skills
   • Stay current with security best practices
   • Contribute to open-source projects and community initiatives
   • Develop cross-functional collaboration capabilities

3. **For Researchers**:
   • Investigate emerging integration patterns
   • Address scalability and performance challenges
   • Explore novel application domains
   • Contribute to standardization efforts

**⚡ Critical Success Factors**:
   • Technical expertise and team capability
   • Robust security and privacy measures
   • Scalable architecture and infrastructure
   • Continuous learning and adaptation"""

        return conclusions

    def _generate_report_footer(self, research_data: List[Dict]) -> str:
        """Generate report footer with metadata"""
        total_time = time.time() - self.start_time if self.start_time else 0

        footer = f"""
{'='*100}
📊 RESEARCH METADATA
{'='*100}
🕒 Total Research Time: {total_time:.1f} seconds
🔍 Search Queries Executed: {len(research_data)}
✅ Successful Retrievals: {sum(1 for data in research_data if data['search_successful'])}
📚 Total Sources Analyzed: {sum(len(data.get('results', [])) for data in research_data)}
⭐ Average Source Quality: {sum(data.get('quality_score', 0) for data in research_data if data.get('search_successful')) / max(sum(1 for data in research_data if data.get('search_successful')), 1):.1f}/10

🤖 Generated by: Advanced ReAct Research Agent v2.0
🔬 AI Models: Gemini 1.5 Flash + Tavily Search API
📅 Report Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S UTC')}

⚠️  DISCLAIMER: This report synthesizes publicly available information as of the generation date.
    Verify critical information through primary sources before making important decisions.
{'='*100}"""

        return footer

    def research_topic_comprehensive(self, topic: str, num_questions: int = 8,
                                   depth_level: str = "comprehensive",
                                   report_format: str = "comprehensive") -> str:
        """
        Main method for comprehensive topic research

        Args:
            topic: Research topic
            num_questions: Number of research questions (1-25)
            depth_level: Research depth ('basic', 'intermediate', 'comprehensive', 'expert')
            report_format: Report format ('brief', 'comprehensive', 'executive')

        Returns:
            Complete research report
        """
        self.start_time = time.time()

        print(f"🚀 STARTING ADVANCED REACT RESEARCH")
        print(f"{'='*80}")
        print(f"📊 Topic: {topic}")
        print(f"❓ Questions: {num_questions}")
        print(f"🎯 Depth: {depth_level}")
        print(f"📋 Format: {report_format}")
        print(f"{'='*80}")

        try:
            # Phase 1: Intelligent Question Generation
            questions = self.generate_smart_research_questions(topic, num_questions, depth_level)

            if not questions:
                return "❌ Error: Could not generate research questions."

            print(f"\n📋 GENERATED RESEARCH QUESTIONS:")
            print(f"{'-'*60}")
            for i, q_data in enumerate(questions, 1):
                print(f"{i:2d}. [{q_data.get('category', 'General')}] {q_data['question']}")
            print()

            # Phase 2: Advanced Web Research
            print(f"🔍 EXECUTING WEB RESEARCH PHASE")
            print(f"{'-'*60}")
            all_research_data = []

            for i, question_data in enumerate(questions, 1):
                print(f"\n📊 Processing Question {i}/{len(questions)}")
                search_result = self.advanced_web_search(question_data)
                all_research_data.append(search_result)

                # Progress indicator
                if search_result['search_successful']:
                    print(f"   ✅ Success | Quality: {search_result['quality_score']:.1f}/10 | Time: {search_result['search_time']:.1f}s")
                else:
                    print(f"   ❌ Failed | Retries: {search_result['retry_count']}")

                # Rate limiting
                time.sleep(self.config.SEARCH_DELAY)

            # Phase 3: Report Generation
            print(f"\n📝 GENERATING COMPREHENSIVE REPORT")
            print(f"{'-'*60}")
            final_report = self.generate_comprehensive_report(topic, all_research_data, report_format)

            # Phase 4: Quality Summary
            successful_searches = sum(1 for data in all_research_data if data['search_successful'])
            total_sources = sum(len(data['results']) for data in all_research_data if data['search_successful'])

            print(f"\n🎉 RESEARCH COMPLETED SUCCESSFULLY!")
            print(f"{'='*60}")
            print(f"✅ Questions Processed: {len(questions)}")
            print(f"🔍 Successful Searches: {successful_searches}")
            print(f"📚 Total Sources: {total_sources}")
            print(f"⏱️  Total Time: {time.time() - self.start_time:.1f} seconds")
            print(f"{'='*60}")

            return final_report

        except Exception as e:
            error_msg = f"❌ Research failed: {str(e)}"
            print(error_msg)
            return error_msg

In [5]:

class ResearchInterface:
    """Enhanced user interface for research operations"""

    def __init__(self, agent: AdvancedReActAgent):
        self.agent = agent
        self.research_history = []

    def display_trending_topics(self):
        """Display categorized trending topics"""
        print("🔥 TRENDING TECHNICAL TOPICS - 2025 EDITION")
        print("="*80)

        topics = TrendingTopics.get_all_topics()

        for i, (category, topic_list) in enumerate(topics.items(), 1):
            print(f"\n{category}")
            print("-" * 60)
            for j, topic in enumerate(topic_list[:5], 1):  # Show top 5 per category
                print(f"   {j}. {topic}")
            if len(topic_list) > 5:
                print(f"   ... and {len(topic_list) - 5} more topics")

        print(f"\n💡 Total Categories: {len(topics)}")
        print(f"📊 Total Topics Available: {sum(len(topics) for topics in topics.values())}")

    def search_topics(self, keyword: str) -> List[Tuple[str, str]]:
        """Search for topics by keyword"""
        results = TrendingTopics.search_topics(keyword)

        if results:
            print(f"🔍 Found {len(results)} topics matching '{keyword}':")
            print("-" * 50)
            for i, (category, topic) in enumerate(results, 1):
                print(f"{i:2d}. [{category.replace('🤖 ', '').replace('🔒 ', '').replace('🚀 ', '')}] {topic}")
        else:
            print(f"❌ No topics found matching '{keyword}'")

        return results

    def get_user_topic_selection(self) -> str:
        """Interactive topic selection"""
        print("\n🎯 TOPIC SELECTION OPTIONS")
        print("="*50)
        print("1. 🔥 Browse trending topics by category")
        print("2. 🔍 Search topics by keyword")
        print("3. ✏️  Enter custom topic")
        print("4. 🎲 Get random trending topic")

        while True:
            try:
                choice = input("\nEnter your choice (1-4): ").strip()

                if choice == "1":
                    return self._browse_by_category()
                elif choice == "2":
                    return self._search_by_keyword()
                elif choice == "3":
                    return self._enter_custom_topic()
                elif choice == "4":
                    random_topics = TrendingTopics.get_random_topics(1)
                    topic = random_topics[0] if random_topics else "Artificial Intelligence"
                    print(f"🎲 Random topic selected: {topic}")
                    return topic
                else:
                    print("❌ Invalid choice. Please enter 1, 2, 3, or 4.")

            except KeyboardInterrupt:
                print("\n👋 Research cancelled by user.")
                return ""
            except Exception as e:
                print(f"❌ Error: {e}")

    def _browse_by_category(self) -> str:
        """Browse topics by category"""
        topics = TrendingTopics.get_all_topics()
        categories = list(topics.keys())

        print(f"\n📂 SELECT CATEGORY:")
        for i, category in enumerate(categories, 1):
            print(f"{i}. {category}")

        try:
            cat_choice = int(input(f"\nEnter category number (1-{len(categories)}): ").strip())
            if 1 <= cat_choice <= len(categories):
                selected_category = categories[cat_choice - 1]
                category_topics = topics[selected_category]

                print(f"\n📊 TOPICS IN {selected_category}:")
                for i, topic in enumerate(category_topics, 1):
                    print(f"{i:2d}. {topic}")

                topic_choice = int(input(f"\nEnter topic number (1-{len(category_topics)}): ").strip())
                if 1 <= topic_choice <= len(category_topics):
                    return category_topics[topic_choice - 1]
                else:
                    print("❌ Invalid topic number.")
                    return category_topics[0]  # Default to first topic
            else:
                print("❌ Invalid category number.")
                return "Artificial Intelligence"  # Default
        except (ValueError, IndexError):
            print("❌ Invalid input. Using default topic.")
            return "Artificial Intelligence"

    def _search_by_keyword(self) -> str:
        """Search topics by keyword"""
        keyword = input("\n🔍 Enter search keyword: ").strip()
        if not keyword:
            return "Artificial Intelligence"

        results = self.search_topics(keyword)
        if not results:
            return keyword  # Use the keyword itself as topic

        try:
            choice = int(input(f"\nSelect topic number (1-{len(results)}): ").strip())
            if 1 <= choice <= len(results):
                return results[choice - 1][1]  # Return the topic name
            else:
                return results[0][1]  # Default to first result
        except (ValueError, IndexError):
            return results[0][1] if results else keyword

    def _enter_custom_topic(self) -> str:
        """Enter custom research topic"""
        topic = input("\n✏️  Enter your research topic: ").strip()
        return topic if topic else "Artificial Intelligence"

    def get_research_parameters(self) -> Dict[str, Any]:
        """Get research parameters from user"""
        print("\n⚙️  RESEARCH CONFIGURATION")
        print("="*50)

        # Number of questions
        try:
            num_questions = int(input("❓ Number of research questions (1-25, default 8): ").strip() or "8")
            num_questions = max(1, min(num_questions, 25))
        except ValueError:
            num_questions = 8

        # Research depth
        print("\n🎯 Research Depth Options:")
        depth_options = {
            "1": "basic",
            "2": "intermediate",
            "3": "comprehensive",
            "4": "expert"
        }

        for key, value in depth_options.items():
            print(f"{key}. {value.title()}")

        depth_choice = input("\nSelect depth level (1-4, default 3): ").strip() or "3"
        depth_level = depth_options.get(depth_choice, "comprehensive")

        # Report format
        print("\n📋 Report Format Options:")
        format_options = {
            "1": "brief",
            "2": "executive",
            "3": "comprehensive"
        }

        for key, value in format_options.items():
            print(f"{key}. {value.title()}")

        format_choice = input("\nSelect report format (1-3, default 3): ").strip() or "3"
        report_format = format_options.get(format_choice, "comprehensive")

        return {
            "num_questions": num_questions,
            "depth_level": depth_level,
            "report_format": report_format
        }


In [6]:

class ReportManager:
    """Manage report generation, saving, and export"""

    @staticmethod
    def save_report(report: str, topic: str, format_type: str = "txt") -> str:
        """Save report to file with timestamp"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        safe_topic = re.sub(r'[^\w\s-]', '', topic).strip().replace(' ', '_')
        filename = f"research_report_{safe_topic}_{timestamp}.{format_type}"

        try:
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(report)

            print(f"💾 Report saved as: {filename}")
            return filename

        except Exception as e:
            print(f"❌ Error saving report: {e}")
            return ""

    @staticmethod
    def download_report(filename: str):
        """Download report file in Colab environment"""
        try:
            files.download(filename)
            print("📥 Download initiated!")
        except Exception as e:
            print(f"❌ Download failed: {e}")
            print("💡 Note: File is saved locally in the environment.")

    @staticmethod
    def create_research_summary(reports: List[str]) -> str:
        """Create summary of multiple research reports"""
        summary_lines = [
            "📊 RESEARCH SESSION SUMMARY",
            "="*60,
            f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
            f"Total Reports: {len(reports)}",
            ""
        ]

        for i, report in enumerate(reports, 1):
            # Extract topic from report (basic parsing)
            lines = report.split('\n')
            topic_line = next((line for line in lines if 'Topic:' in line), f"Report {i}")
            summary_lines.append(f"{i}. {topic_line}")

        return '\n'.join(summary_lines)


In [7]:

def main():
    """Enhanced main application with full user interface"""

    print("🤖 ADVANCED REACT WEB RESEARCH AGENT - 2025 EDITION")
    print("="*80)
    print("🚀 Enhanced AI-Powered Technical Research Platform")
    print("⚡ Powered by Gemini AI + Tavily Search + Advanced Analytics")
    print("="*80)

    # Initialize configuration
    config = Config()

    if not config.GEMINI_API_KEY or not config.TAVILY_API_KEY:
        print("❌ ERROR: Missing API keys!")
        print("Please set your API keys in Google Colab secrets:")
        print("1. GOOGLE_API_KEY (for Gemini)")
        print("2. TAVILY_API_KEY (for web search)")
        return

    try:
        # Initialize agent and interface
        agent = AdvancedReActAgent(config)
        interface = ResearchInterface(agent)
        report_manager = ReportManager()

        session_reports = []

        while True:
            print(f"\n🎯 RESEARCH MENU")
            print("-"*40)
            print("1. 🔍 Start New Research")
            print("2. 🔥 Browse Trending Topics")
            print("3. 📊 View Research History")
            print("4. 💾 Download All Reports")
            print("5. 🚪 Exit")

            try:
                choice = input("\nEnter your choice (1-5): ").strip()

                if choice == "1":
                    # Start new research
                    print(f"\n🚀 STARTING NEW RESEARCH SESSION")
                    print("="*60)

                    # Get topic selection
                    topic = interface.get_user_topic_selection()
                    if not topic:
                        continue

                    # Get research parameters
                    params = interface.get_research_parameters()

                    # Confirm research parameters
                    print(f"\n📋 RESEARCH CONFIGURATION SUMMARY")
                    print("-"*50)
                    print(f"📊 Topic: {topic}")
                    print(f"❓ Questions: {params['num_questions']}")
                    print(f"🎯 Depth: {params['depth_level']}")
                    print(f"📋 Format: {params['report_format']}")

                    confirm = input("\n🚀 Start research? (y/n): ").strip().lower()
                    if confirm not in ['y', 'yes']:
                        print("❌ Research cancelled.")
                        continue

                    # Execute research
                    print(f"\n🔬 EXECUTING RESEARCH...")
                    report = agent.research_topic_comprehensive(
                        topic=topic,
                        num_questions=params['num_questions'],
                        depth_level=params['depth_level'],
                        report_format=params['report_format']
                    )

                    # Display report
                    print(f"\n📊 RESEARCH REPORT")
                    print("="*80)
                    print(report)

                    # Save report
                    save_choice = input(f"\n💾 Save report to file? (y/n): ").strip().lower()
                    if save_choice in ['y', 'yes']:
                        filename = report_manager.save_report(report, topic)
                        if filename:
                            download_choice = input("📥 Download now? (y/n): ").strip().lower()
                            if download_choice in ['y', 'yes']:
                                report_manager.download_report(filename)

                    session_reports.append(report)

                elif choice == "2":
                    # Browse trending topics
                    interface.display_trending_topics()

                elif choice == "3":
                    # View research history
                    if session_reports:
                        print(f"\n📚 RESEARCH HISTORY ({len(session_reports)} reports)")
                        print("-"*50)
                        for i, report in enumerate(session_reports, 1):
                            # Extract topic from report title
                            lines = report.split('\n')
                            topic_line = next((line for line in lines if 'Topic:' in line), f"Report {i}")
                            print(f"{i}. {topic_line}")
                    else:
                        print("\n📭 No research reports in current session.")

                elif choice == "4":
                    # Download all reports
                    if session_reports:
                        summary = report_manager.create_research_summary(session_reports)
                        summary_filename = report_manager.save_report(summary, "Session_Summary")
                        if summary_filename:
                            report_manager.download_report(summary_filename)
                    else:
                        print("\n📭 No reports to download.")

                elif choice == "5":
                    # Exit
                    print(f"\n👋 Thank you for using Advanced ReAct Research Agent!")
                    print(f"📊 Session Summary: {len(session_reports)} reports generated")
                    print("🚀 Happy researching!")
                    break

                else:
                    print("❌ Invalid choice. Please enter 1-5.")

            except KeyboardInterrupt:
                print(f"\n\n👋 Research session interrupted by user.")
                break
            except Exception as e:
                print(f"❌ Error: {e}")
                print("Please try again or contact support.")

    except Exception as e:
        print(f"❌ Fatal error initializing application: {e}")
        print("Please check your configuration and try again.")

In [8]:

def quick_research(topic: str = None, questions: int = 5):
    """Quick research function for immediate use"""
    config = Config()

    if not config.GEMINI_API_KEY or not config.TAVILY_API_KEY:
        print("❌ Please set your API keys first!")
        return

    if not topic:
        topic = TrendingTopics.get_random_topics(1)[0]
        print(f"🎲 Random topic selected: {topic}")

    try:
        agent = AdvancedReActAgent(config)
        report = agent.research_topic_comprehensive(topic, questions, "comprehensive", "comprehensive")

        print("\n" + "="*80)
        print("📊 QUICK RESEARCH COMPLETED")
        print("="*80)
        print(report)

        return report

    except Exception as e:
        print(f"❌ Quick research failed: {e}")
        return None

def demo_trending_topics():
    """Demonstrate trending topics functionality"""
    print("🔥 TRENDING TOPICS DEMO")
    print("="*50)

    # Show all categories
    topics = TrendingTopics.get_all_topics()
    for category, topic_list in topics.items():
        print(f"\n{category}")
        print(f"   Sample: {topic_list[0]}")

    # Demo search
    print(f"\n🔍 SEARCH DEMO:")
    search_results = TrendingTopics.search_topics("AI")
    print(f"Found {len(search_results)} topics containing 'AI'")

In [ ]:

if __name__ == "__main__":
    # Uncomment the function you want to run:

    # Full interactive application
    main()

    # Quick research demo
    # quick_research("Quantum Computing", 6)

    # Trending topics demo
    # demo_trending_topics()

    print("\n✅ Script execution completed!")

🤖 ADVANCED REACT WEB RESEARCH AGENT - 2025 EDITION
🚀 Enhanced AI-Powered Technical Research Platform
⚡ Powered by Gemini AI + Tavily Search + Advanced Analytics
✅ Advanced ReAct Agent initialized successfully!
📊 Model: gemini-1.5-flash-latest
🔍 Max search results per query: 5

🎯 RESEARCH MENU
----------------------------------------
1. 🔍 Start New Research
2. 🔥 Browse Trending Topics
3. 📊 View Research History
4. 💾 Download All Reports
5. 🚪 Exit

Enter your choice (1-5): 1

🚀 STARTING NEW RESEARCH SESSION

🎯 TOPIC SELECTION OPTIONS
1. 🔥 Browse trending topics by category
2. 🔍 Search topics by keyword
3. ✏️  Enter custom topic
4. 🎲 Get random trending topic

Enter your choice (1-4): 1

📂 SELECT CATEGORY:
1. 🤖 AI & Machine Learning
2. 🔒 Cybersecurity & Privacy
3. 🚀 Emerging Technologies
4. ☁️ Cloud & Infrastructure
5. 💰 Fintech & Blockchain
6. 🏥 HealthTech & BioTech
7. 🌐 Web & Mobile Development
8. 📊 Data Science & Analytics

Enter category number (1-8): 4

📊 TOPICS IN ☁️ Cloud & Infrast

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Download initiated!

🎯 RESEARCH MENU
----------------------------------------
1. 🔍 Start New Research
2. 🔥 Browse Trending Topics
3. 📊 View Research History
4. 💾 Download All Reports
5. 🚪 Exit

Enter your choice (1-5): 1

🚀 STARTING NEW RESEARCH SESSION

🎯 TOPIC SELECTION OPTIONS
1. 🔥 Browse trending topics by category
2. 🔍 Search topics by keyword
3. ✏️  Enter custom topic
4. 🎲 Get random trending topic
